In [10]:
import os
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
sns.set()

## Data Processing

In [12]:
# data paths
data_dir = os.path.join(os.getcwd(), 'home-credit-default-risk')
test_path = os.path.join(data_dir, 'application_test.csv')
train_path = os.path.join(data_dir, 'application_train.csv')

In [13]:
# load DataFrames
df_train = pd.read_csv(train_path)
df_train.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/liamkelly/credit-loan-worthiness/home-credit-default-risk/application_train.csv'

In [ ]:
df_train.shape

In [ ]:
df_train.columns

In [ ]:
df_train['TARGET'].mean()

A `1` in the `TARGET` column indicates that the client defaulted

In [ ]:
# replace {Y, N} with {1, 0}
for column in df_train.columns:
    if df_train[column].dtype == 'object':
        df_train[column] = df_train[column].replace({'Y': 1, 'N': 0})

In [ ]:
# split numerical and categorical data
numerical_cols = df_train.select_dtypes(include=['number']).columns
categorical_cols = df_train.select_dtypes(exclude=['number']).columns

# impute missing values based on mean
num_imp = SimpleImputer(strategy='mean')
num_data_imp = num_imp.fit_transform(df_train[numerical_cols])
num_df = pd.DataFrame(num_data_imp, columns=numerical_cols)

# impute missing values based on most frequent
cat_imp = SimpleImputer(strategy='most_frequent')
cat_data_imp = cat_imp.fit_transform(df_train[categorical_cols])
cat_df = pd.DataFrame(cat_data_imp, columns=categorical_cols)

# concatenate back into complete dataframe
df_train_clean = pd.concat([num_df, cat_df], axis=1)

In [ ]:
# choose the top 10 highest correlation columns (positive or negative)
abs_corrs = df_train_clean.corr()['TARGET'].abs().sort_values(ascending=False)[1:11]
top_corrs = df_train_clean.corr()['TARGET'].loc[abs_corrs.index]

# plot the results
plt.figure(figsize=(10, 6))
sns.barplot(x=top_corrs.index, y=top_corrs.values, palette="coolwarm")
plt.title("Top 10 Highest Correlations to Loan Defaulting")
plt.xlabel("Feature")
plt.ylabel("Correlation with Target")
plt.xticks(rotation=45)
plt.show()

`EXT_SOURCE_1`, `EXT_SOURCE_2`, and `EXT_SOURCE_3` are credit scores from external sources

## SMOTE

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

X = df_train_clean.drop('TARGET', axis=1)  # Drop the target column to separate features
y = df_train_clean['TARGET']  # Separate the target

# Splitting the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features (optional based on model requirements)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Combining the resampled data back into a new dataframe
import pandas as pd
df_train_smote = pd.DataFrame(X_train_smote, columns=X_train.columns)
df_train_smote['TARGET'] = y_train_smote  # Adding the target column back

df_test_clean = pd.DataFrame(X_test_scaled, columns=X_test.columns)
df_test_clean['TARGET'] = y_test

## Find the Best Model

In [ ]:
# find the columns that have the strongest correlations
cols = list(top_corrs.index)

# filter out only the strongest correlation columns and target
X = df_train_clean[cols]
y = df_train_clean['TARGET']

In [ ]:
# split into training and validation set to find best performing model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Random Forest

In [ ]:
# create and fit Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rf_clf.fit(X_train, y_train)

# find training and validation accuracy scores
print(f'Training Accuracy: {rf_clf.score(X_train, y_train):.4f}')
print(f'Test Accuracy: {rf_clf.score(X_test, y_test):.4f}')

In [ ]:
# y prediction values
y_pred_rf=(rf_clf.predict(X_test))

# create and plot confusion matrix
cm = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix - Random Forest Classifier")
plt.show()

### Gradient Boosting

In [ ]:
# create and fit GB classifier
gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(X_train, y_train)

# find training and validation accuracy scores
print(f'Training Accuracy: {gb_clf.score(X_train, y_train):.4f}')
print(f'Test Accuracy: {gb_clf.score(X_test, y_test):.4f}')

In [ ]:
# y prediction values
y_pred_gb=(gb_clf.predict(X_test))

# create and plot confusion matrix
cm = confusion_matrix(y_test, y_pred_gb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix - Gradient Boosting Classifier")
plt.show()

### Extreme Gradient Boosting

In [ ]:
# create and fit XGB classifier
xgb_clf = XGBClassifier(random_state=42, max_depth=3, learning_rate=.1)
xgb_clf.fit(X_train, y_train)

# find training and validation accuracy scores
print(f'Training Accuracy: {xgb_clf.score(X_train, y_train):.4f}')
print(f'Test Accuracy: {xgb_clf.score(X_test, y_test):.4f}')

In [ ]:
# y prediction values
y_pred_xgb=(xgb_clf.predict(X_test))

# create and plot confusion matrix
cm = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix - Extreme Gradient Boosting Classifier")
plt.show()

## Neural Network

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

X = df_train_clean[cols]  
y = df_train_clean['TARGET']  

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# MLP Classifier setup
mlp = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', solver='adam', max_iter=300, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Making predictions and evaluating the model
y_pred = mlp.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(report)

## Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = df_train_clean[cols]
y = df_train_clean['TARGET']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and fit the logistic regression model
log_reg = LogisticRegression(max_iter=300, random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Making predictions
y_pred = log_reg.predict(X_test_scaled)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

## Choosing a Model

Based on the extremely similar scores coming from each of the Ensemble Methods models, we decided to choose the Random Forest Classifier as the model that would best predict whether a client would default or not. This is because out of all of the classifiers, it had the fewest number of false negatives (Predicted the client would not default, but they did). We chose this due to the nature of the problem. Predicting a client is reliable and approving them when they actually aren't has far worse implications than falsely rejecting an application. 

## Predicting

### Pre-Processing

In [ ]:
df_test = pd.read_csv(test_path)

In [ ]:
# replace {Y, N} with {1, 0}
for column in df_test.columns:
    if df_test[column].dtype == 'object':
        df_test[column] = df_test[column].replace({'Y': 1, 'N': 0})

In [ ]:
# split numerical and categorical data
numerical_cols = df_test.select_dtypes(include=['number']).columns
categorical_cols = df_test.select_dtypes(exclude=['number']).columns

# impute missing values based on mean
num_imp = SimpleImputer(strategy='mean')
num_data_imp = num_imp.fit_transform(df_test[numerical_cols])
num_df = pd.DataFrame(num_data_imp, columns=numerical_cols)

# impute missing values based on most frequent
cat_imp = SimpleImputer(strategy='most_frequent')
cat_data_imp = cat_imp.fit_transform(df_test[categorical_cols])
cat_df = pd.DataFrame(cat_data_imp, columns=categorical_cols)

# concatenate back into complete dataframe
df_test_clean = pd.concat([num_df, cat_df], axis=1)

In [ ]:
# train new classifier on the entire training dataset
test_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
test_clf.fit(X, y)

`X` and `y` are the cleaned training dataframe filtered for only the strongest correlation columns.

In [ ]:
# filter out only the strongest correlation columns
X_test_pred = df_test_clean[cols]

In [ ]:
# predict on the test data
y_pred_test=(test_clf.predict(X_test_pred))

In [ ]:
# find the 1 predicted values
y_pred_true = y_pred_test[y_pred_test == 1]

# find the percentage of predicted 1s
pred_percentage = (len(y_pred_true) / len(y_pred_test)) * 100

In [ ]:
# find the 1 true values
y_true = y[y == 1]

# find the percentage of true 1s
true_percentage = (len(y_true) / len(y)) * 100

In [ ]:
print(f'Actual Percentage: {true_percentage}')
print(f'Predicted Percentage: {pred_percentage}')

In [ ]:
labels = ['Actual Default Percentage', 'Predicted Default Percentage']
values = [true_percentage, pred_percentage]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
colors = ['slateblue', 'darkblue']

# plot first subplot with percentage range 0-100
axs[0].bar(labels, values, color=colors)
for bar, value in zip(axs[0].patches, values):
    axs[0].text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2f}%', ha='center', va='bottom')
axs[0].set_ylabel('Percentage')
axs[0].set_title('Comparison of True and Predicted Percentages of Positive Labels (1s)')
axs[0].set_ylim(0, 100)

# plot second  subplot with percentage range 0-10
axs[1].bar(labels, values, color=colors)
for bar, value in zip(axs[1].patches, values):
    axs[1].text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2f}%', ha='center', va='bottom')
axs[1].set_ylabel('Percentage')
axs[0].set_title('Comparison of True and Predicted Percentage of Default Labels')
axs[1].set_ylim(0, 10)

plt.tight_layout()
plt.show()

This is a very interesting relationship. It appears the models are trained to prioritize avoiding false negatives over false positives for the sake of accuracy. However, in this situation, it would be beneficial to predict a higher number of true positives

In [ ]:
print(f'Precision (training data, test set): {precision_score(y_test, y_pred_rf)}')
print(f'Recall (training data, test set): {recall_score(y_test, y_pred_rf)}')
print(f'F1 Score (training data, test set): {f1_score(y_test, y_pred_rf)}')

These suspicions were confirmed with the shockingly low precision, recall, and F1 scores of the training data

### Threshold Testing

In order to avoid a high accuracy, low effectiveness model, we next experimented with thresholds. We tested each threshold from .05 to .4 and found which one would return the least false positives while also maintaining high accuracy.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

thresholds = np.arange(0.05, 0.45, 0.05)
confusion_matrices = []
accuracies = []
recalls = []
precisions = []
f1s = []
tps = []
fps = []
tns = []
fns = []

In [ ]:
# loop through each threshold
for threshold in thresholds:
    # initialize and train the RandomForestClassifier
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(X_train, y_train)
    
    # predict probabilities and compare to threshold
    y_pred_prob = rf_classifier.predict_proba(X_test)[:, 1]
    y_pred_adjusted = (y_pred_prob > threshold).astype(int)
    
    # calculate confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_adjusted)
    confusion_matrices.append(conf_matrix)
    
    # fetch all values from confusion matrix
    tp = conf_matrix[1, 1]
    fp = conf_matrix[0, 1]
    tn = conf_matrix[0, 0]
    fn = conf_matrix[1, 0]
    tps.append(tp)
    fps.append(fp)
    tns.append(tn)
    fns.append(fn)
    
    # calculate and append scores
    acc = accuracy_score(y_test, y_pred_adjusted)
    accuracies.append(acc)
    rec = recall_score(y_test, y_pred_adjusted)
    recalls.append(rec)
    prec = precision_score(y_test, y_pred_adjusted)
    precisions.append(prec)
    f1 = f1_score(y_test, y_pred_adjusted)
    f1s.append(f1)

# store results in DF
results_df = pd.DataFrame({
    'threshold': thresholds,
    'confusion_matrix': confusion_matrices,
    'accuracy': accuracies,
    'recall': recalls,
    'precision': precisions,
    'F1': f1s,
    'TP': tps,
    'FP': fps,
    'TN': tns,
    'FN': fns
})

In [ ]:
# create subplots for each threshold
fig, axs = plt.subplots(3, 3, figsize=(15, 15))

for i, threshold in enumerate(thresholds):
    row = i // 3
    col = i % 3
    cm = confusion_matrices[i]
    axs[row, col].imshow(cm, cmap='Blues', interpolation='nearest')
    axs[row, col].set_title(f'Threshold: {threshold}:')
    axs[row, col].set_xlabel('Predicted labels')
    axs[row, col].set_ylabel('True labels')
    for i in range(len(cm)):
        for j in range(len(cm)):
            axs[row, col].text(j, i, cm[i, j], ha='center', va='center', color='black')

plt.tight_layout()
plt.show()

In [ ]:
results_df

Based on the data, we chose .1 as a reasonable decision threshold that increases recall and f1 without completely tanking accuracy.

In [ ]:
conf_matrix_1 = results_df['confusion_matrix'][1]
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_1, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix - Random Forest Classifier (Decision Threshold .1)")
plt.show()

In [ ]:
# predict with the new decision threshold
threshold = .1
test_pred_prob = test_clf.predict_proba(X_test_pred)[:, 1]
y_pred_test = (test_pred_prob > .1).astype(int)

In [ ]:
# find the 1 predicted values
y_pred_true = y_pred_test[y_pred_test == 1]

# find the percentage of predicted 1s
pred_percentage = (len(y_pred_true) / len(y_pred_test)) * 100

In [ ]:
print(f'Actual Percentage: {true_percentage}')
print(f'Predicted Percentage: {pred_percentage}')

In [ ]:
labels = ['Actual Default Percentage', 'Predicted Default Percentage']
values = [true_percentage, pred_percentage]

# plot bars and add labels
plt.figure(figsize=(8, 6))
bars = plt.bar(labels, values, color=colors)
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{value:.2f}%', ha='center', va='bottom')

plt.ylabel('Percentage')
plt.title('Comparison of True and Predicted Percentage of Default Labels (Decision Threshold .1)')
plt.ylim(0, 50)
plt.show()


While the rate of positive (default) classifications is now much higher than the actual default percentage, this is preferred because we would much rather have the program incorrectly classify an applicant as high risk than mistakenly let a risky applicant through who later defaults on the loan.

The model does not currently work well. An increase in correlated features may improve it in the future.